# Graphic Statics IV: Thruss

reference: 
https://block.arch.ethz.ch/brg/files/2012-jiass-vanmele-lachauer-rippmann-block_1380094579.pdf

In [ ]:
import math as m

import compas
import compas_ags

from compas_ags.diagrams import FormGraph
from compas_ags.diagrams import FormDiagram
from compas_ags.diagrams import ForceDiagram
from compas_ags.ags import graphstatics
from compas_ags.viewers import Viewer

import warnings
warnings.filterwarnings("ignore")

###    create a truss

In [ ]:
### 
###  INPUT HOW MANY LOADS TO ADD
###  INPUT SPAN LENGTH
###
loads = 5
span = 10

In [ ]:
def catenary_curve(x, type='hanging_cable', a=5, xm=0, ym=0):
    if type is 'arch':
        return ym - a * m.cosh((x - xm) / a)
    elif type is 'hanging_cable':
        return ym + a * m.cosh((x - xm) / a)

In [ ]:
nodes = []
edges = []

# to save the pair of top and bottom chords
chord_pairs = []

# truss
y0 = catenary_curve(- span / 2)
for i in range(loads + 2):
    x = i * span / (loads + 1) - span / 2 
    nodes.append([x, y0, 0])
    if i == 0: 
        edges.append(tuple([i, i+1]))
        edges.append(tuple([i, i+2])) 
        chord_pairs.append([tuple([i, i+1]), tuple([i, i+2])])
    elif i == loads:
        nodes.append([x, catenary_curve(x), 0])
        edges.append(tuple([i * 2 -1, i * 2 + 1]))
        edges.append(tuple([i * 2 - 1, i * 2]))
        edges.append(tuple([i * 2, i * 2 + 1]))
        chord_pairs.append([tuple([i * 2 -1, i * 2 + 1]), tuple([i * 2, i * 2 + 1])])
    elif i != loads + 1:
        nodes.append([x, catenary_curve(x), 0])
        edges.append(tuple([i * 2 - 1, i * 2 + 1]))
        edges.append(tuple([i * 2 - 1, i * 2]))
        edges.append(tuple([i * 2, i * 2 + 2]))
        chord_pairs.append([tuple([i * 2 -1, i * 2 + 1]), tuple([i * 2, i * 2 + 2])])
        
# vertical reaction force
for i in range(loads + 2):
    x = i * span / (loads + 1) - span / 2 
    if i == 0:
        nodes.append([x, y0 - 1, 0])
        edges.append(tuple([0, len(nodes) - 1]))
    elif i == loads + 1:
        nodes.append([x, y0 - 1, 0])
        edges.append(tuple([i * 2 - 1, len(nodes) - 1]))
    else:
        nodes.append([x, y0 + 1, 0])
        edges.append(tuple([i * 2 - 1, len(nodes) - 1]))
    
# horizontal reaction force
nodes.append([-span / 2 - 1.0, y0, 0])
edges.append(tuple([0, len(nodes) - 1]))
nodes.append([span / 2 + 1.0, y0, 0])
edges.append(tuple([2 * loads + 1, len(nodes) - 1]))

In [ ]:
# make form and force diagrams
graph = FormGraph.from_nodes_and_edges(nodes, edges)
form = FormDiagram.from_graph(graph)
force = ForceDiagram.from_formdiagram(form)

###    prescribe edge force density and set fixed vertices

In [ ]:
# set the magnitude of the applied load
e =  {'v': list(form.vertices_where({'x': - span / 2, 'y': y0}))[0],
       'u': list(form.vertices_where({'x': - span / 2 , 'y': y0-1.0}))[0]}
form.edge_attribute((e['u'], e['v']), 'is_ind', True)
form.edge_attribute((e['u'], e['v']), 'q', -5.)

e =  {'v': list(form.vertices_where({'x': - span / 2, 'y': y0}))[0],
       'u': list(form.vertices_where({'x': - span / 2 -1 , 'y':y0}))[0]}
form.edge_attribute((e['u'], e['v']), 'is_ind', True)
form.edge_attribute((e['u'], e['v']), 'q', 0.)

# set the fixed points
left = list(form.vertices_where({'x': - span / 2, 'y': y0}))[0]
right = list(form.vertices_where({'x': span / 2, 'y': y0}))[0]
fixed = [left, right]
form.set_fixed(fixed)
force.set_anchor([0])

# update the diagrams
graphstatics.form_update_q_from_qind(form)
graphstatics.force_update_from_form(force, form)

### display force and form diagrams

In [ ]:
# store lines representing the current state of equilibrium
form_lines = []
for u, v in form.edges():
    form_lines.append({
        'start': form.vertex_coordinates(u, 'xy'),
        'end'  : form.vertex_coordinates(v, 'xy'),
        'width': 1.0,
        'color': '#cccccc',
        'style': '--'
    })

force_lines = []
for u, v in force.edges():
    force_lines.append({
        'start': force.vertex_coordinates(u, 'xy'),
        'end'  : force.vertex_coordinates(v, 'xy'),
        'width': 1.0,
        'color': '#cccccc',
        'style': '--'
    })

In [ ]:
viewer = Viewer(form, force, delay_setup=False)

viewer.draw_form(
    arrows_on=True,
    vertexsize=0.15,
    vertexlabel={key: key for key in form.vertices()},
    edgelabel={uv: index for index, uv in enumerate(form.edges())},)

    
viewer.draw_force(
    arrows_on=True,
    vertexsize=0.15,
    vertexlabel={key: key for key in force.vertices()},
    edgelabel=viewer.check_edge_pairs()[1])

viewer.show()

### modify force in the bottom chord

In [ ]:
from compas_ags.ags2.constraints import ConstraintsCollection, HorizontalFix, VerticalFix
import compas_ags.ags2.rootfinding as rf
import numpy as np

# set constraints
# find vertices in the force diagram and move them to the right
# which means making the internal forces and boundary forces smaller
C = ConstraintsCollection(form)
C.add_constraint(HorizontalFix(form, left))
C.add_constraint(VerticalFix(form, left))
C.add_constraint(HorizontalFix(form, right))
for i in range(loads):
    C.add_constraint(HorizontalFix(form, i * loads + 1))
C.constrain_dependent_leaf_edges_lengths()

# modify the geometry of the force diagram and update the form diagram using Newton's method
xy = np.array(form.xy(), dtype=np.float64).reshape((-1, 2))
_xy = np.array(force.xy(), dtype=np.float64).reshape((-1, 2))

In [ ]:
from compas.geometry import distance_point_point
from itertools import chain

form_edges = list(form.edges())
top_indices = [form_edges.index(pairs[0]) for pairs in chord_pairs]
bottom_indices = [form_edges.index(pairs[1]) for pairs in chord_pairs]
force_idx_uv = {idx:uv for uv, idx in viewer.check_edge_pairs()[1].items()}

force_bottom = [force_idx_uv[bottom] for bottom in bottom_indices]
force_top = [force_idx_uv[top] for top in top_indices]

# find the center of the circle
cen_idx = max(set(chain.from_iterable(force_bottom)), key=force_bottom.count) 

# if the target force / radius is not given
if True:
    radius = 0
    for (u, v) in force_bottom:
        xy_u = force.vertex_coordinates(u)
        xy_v = force.vertex_coordinates(v)
        dis = distance_point_point(xy_u, xy_v)
        if dis > radius: radius = dis 

for i, (bottom, top) in enumerate(zip(force_bottom, force_top)):
    common_idx = list(set(bottom).intersection(top))
    cathetus_a = distance_point_point(force.vertex_coordinates(top[0]), force.vertex_coordinates(top[1]))
    catetus_b_pt = list(set(top) - set(common_idx))[0]
    cathetus_b = distance_point_point(force.vertex_coordinates(cen_idx), force.vertex_coordinates(catetus_b_pt))
    x_dis = m.sqrt(radius ** 2 - cathetus_b ** 2) - cathetus_a
    _xy[force.key_index()[common_idx[0]], 0] -= x_dis
    
    print(i, bottom, top, common_idx)

In [ ]:
# compute the form diagram
_X_goal = np.vstack((np.asmatrix(_xy[:, 0]).transpose(), np.asmatrix(_xy[:, 1]).transpose()))
rf.compute_form_from_force_newton(form, force, _X_goal, constraints=C)

In [ ]:
viewer = Viewer(form, force, delay_setup=False)

viewer.draw_form(lines=form_lines,
                 forces_on=True,
                 vertexlabel={key: key for key in form.vertices()},
                 external_on=False,
                 vertexsize=0.2,
                 vertexcolor={key: '#000000' for key in fixed},
                 edgelabel={uv: index for index, uv in enumerate(form.edges())}
)

viewer.draw_force(lines=force_lines,
                  vertexlabel={key: key for key in force.vertices()},
                  vertexsize=0.2,
                  edgelabel=viewer.check_edge_pairs()[1]
)

viewer.show()